In [66]:
import gensim
from collections import namedtuple
from pathlib import Path
import re
import numpy as np

HAMS_SIZE = 3672
SPAMS_SIZE = 1500

In [40]:
file_path = "./enron1/ham/"

ham_tmp = []
path = Path(file_path if len(file_path) >= 2 else '.')
for path_in in [x for x in path.glob('*.txt') if x.is_file()]:
    with path_in.open() as f:
        ham_tmp.append(f.read())
        
file_path = "./enron1/spam/"

spam_tmp = []
path = Path(file_path if len(file_path) >= 2 else '.')
for path_in in [x for x in path.glob('*.txt') if x.is_file()]:
    with path_in.open(encoding = "ISO-8859-1") as f:
        spam_tmp.append(f.read())

In [48]:
ham_docs = []
spam_docs = []
docs = []

analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i,text in enumerate(ham_tmp):
    text = re.sub(re.compile("[!-/:-@[-`{-~]"), '', text)
    text = re.sub(re.compile("\n"), '', text)
    ham_docs.append(analyzedDocument(text.split(),[i]))
    docs.append(analyzedDocument(text.split(),[i]))
    
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i,text in enumerate(spam_tmp):
    text = re.sub(re.compile("[!-/:-@[-`{-~]"), '', text)
    text = re.sub(re.compile("\n"), '', text)
    spam_docs.append(analyzedDocument(text.split(),[i]))
    docs.append(analyzedDocument(text.split(),[i+len(ham_docs)]))

In [50]:
model = gensim.models.doc2vec.Doc2Vec(docs, min_count=0)

In [54]:
data = []
target = []
for i in range(HAMS_SIZE + SPAMS_SIZE):
    data.append(list(model.docvecs[i]))
    if i < HAMS_SIZE:
        target.append(0)
    else:
        target.append(1)

In [79]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve,f1_score
import lightgbm as lgb
import xgboost as xgb
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

In [70]:
# ここから学習をする．
# clf1:giniを使ったRF
# clf2:entropyを使ったRF
# clf3:giniを使ったET
# clf4:entropyを使ったET
# clf5:適当にパラメータを決定したGBDT
# clf6:gridSearchでGBDTをがっつり検証した結果
# clf7:gridSearchでGBDTをゆるふわ検証した結果
# clf8:LightGBMで適当に
# clf9:LightGBMをチューニング
# clf10:XGBoostで適当に
# clf11:XGBoostをチューニング
# clf12:KNN
# clf13:Naive Bayes
# clf14:svc

lr = LogisticRegression()
clf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini')
clf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy')
clf3 = ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini')
clf4 = ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy')
clf5 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1, max_depth=3)
clf6 = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=2,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=9,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100000, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)
clf7 = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=2,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=9,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=3000, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)
clf8 = lgb.LGBMClassifier(objective='binary',
                        num_leaves = 23,
                        learning_rate=0.1,
                        n_estimators=100)
clf9 = lgb.LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.63437546541680478,
        learning_rate=0.1, max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=1375,
        nthread=-1, num_leaves=316, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=0, silent=True, subsample=0.97178964806068002,
        subsample_for_bin=50000, subsample_freq=1)
clf10 = xgb.XGBClassifier(objective='binary:logistic',
                        max_depth=5,
                        learning_rate=0.1,
                        n_estimators=100)
clf11 = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.91837825878391688, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=3000, nthread=-1, objective='binary:logistic',
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.81182984013631532)
clf12 = KNeighborsClassifier(n_neighbors=1)
clf13 = GaussianNB()
clf14 = SVC()

In [76]:
sclf = StackingCVClassifier(classifiers=[clf1,clf2,clf4,clf5,clf7,clf8,clf9,clf10,clf11],use_probas=True,meta_classifier=lr)

print('4-fold cross validation:\n')

for clf, label in zip([clf1,clf2,clf4,clf5,clf7,clf8,clf9,clf10,clf11,sclf], 
                      ['RF_gini', 
                       'RF_entropy', 
                       'ET_entropy',
                       'GBDT_pure',
                       'GBDT_grid',
                       'LGBM_pure',
                       'LGBM_grid',
                       'XGBOOST_pure',
                       'XGBOOST_grid',
                       'StackingClassifier']):

    scores = cross_val_score(clf, np.array(data), np.array(target), 
                            cv=4, scoring='f1_macro')
    print("f1_macro: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

4-fold cross validation:

f1_macro: 0.78 (+/- 0.02) [RF_gini]
f1_macro: 0.79 (+/- 0.03) [RF_entropy]
f1_macro: 0.78 (+/- 0.03) [ET_entropy]
f1_macro: 0.78 (+/- 0.03) [GBDT_pure]
f1_macro: 0.81 (+/- 0.02) [GBDT_grid]
f1_macro: 0.81 (+/- 0.02) [LGBM_pure]
f1_macro: 0.82 (+/- 0.02) [LGBM_grid]
f1_macro: 0.81 (+/- 0.02) [XGBOOST_pure]
f1_macro: 0.82 (+/- 0.02) [XGBOOST_grid]
f1_macro: 0.82 (+/- 0.03) [StackingClassifier]


In [92]:
X_train, X_test,y_train,y_test = train_test_split(np.array(data),np.array(target),test_size=0.33)

binary_model = sclf.fit(X_train,y_train)

y_pred = binary_model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["ham", "spam"]))

             precision    recall  f1-score   support

        ham       0.88      0.94      0.91      1219
       spam       0.82      0.67      0.74       488

avg / total       0.86      0.86      0.86      1707



In [93]:
scores

array([ 0.80803818,  0.84524474,  0.84495545,  0.78610649])